In [3]:
from numpy import exp, array, random, dot
import numpy as np
from scipy import misc
import skimage
import math
from __future__ import with_statement

In [4]:
arr1 = np.divide(misc.imread('tAmerica1.png'),255)
arr1r = arr1[:,:,0]
arr1g = arr1[:,:,1]
arr1b = arr1[:,:,2]
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [30]:
filterNum = 1
filterSize = 10
layerNum = 6
epochs = 10

In [26]:
filters = 0.01*random.random((filterNum*layerNum, filterSize, filterSize))
print(filters)

[[[  1.65004509e-04   8.50937566e-03   3.34649418e-03   1.33167706e-03
     1.59001859e-03   3.39474774e-04   5.35337820e-03   8.98443897e-03
     8.66819837e-03   5.11208294e-03]
  [  8.17076277e-03   9.75287396e-03   9.17951768e-03   8.22883584e-04
     6.08609642e-03   8.21383371e-03   6.55662755e-05   8.80094102e-03
     6.61723934e-04   4.90311082e-04]
  [  5.40647945e-03   1.47768381e-03   5.48845345e-05   8.40940252e-03
     6.46627215e-03   2.58783667e-03   9.94520437e-03   9.32094036e-03
     1.42683094e-03   8.12099942e-03]
  [  5.23146691e-03   5.41764597e-03   8.39404203e-03   9.43508093e-03
     8.00884237e-04   4.96310018e-03   3.63826665e-03   4.95467236e-03
     6.42150166e-03   1.55118319e-03]
  [  1.70536725e-03   5.76330486e-03   8.20466339e-03   2.04702841e-03
     5.92128344e-03   7.31728995e-03   2.71421781e-03   9.69529850e-03
     7.41913035e-03   9.19428436e-03]
  [  6.47119943e-03   4.97313211e-04   6.55482007e-03   8.36284651e-03
     2.90891211e-03   4.73280

In [7]:
def activation(x, deriv=False):
    if deriv:
        return (x > 0)
    return x * (x > 0)
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
def apply(filterArr, img, row, col):
    value = 0
    halfFil = int(filterSize/2)
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0))
    for y in range(len(filterArr)):
        for x in range(len(filterArr[0])):
            value += padImg[y+row-halfFil][x+col-halfFil]*filterArr[y][x]
    return value/sumNoNeg(filterArr)
print(apply(filters[4],arr1r,0,0))

0.0


In [8]:
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])):
            new_img[y][x] = activation(apply(filterArr,img,y,x))
    return new_img
# print(len(([[0]*(len(arr1r))]*(len(arr1r[0])))[0]))
# print(len(arr1r[0]))
# #print([[0]*(len(arr1r)-filterSize)]*(len(arr1r[0])-filterSize))
# print(len(arr1r))
# print(len(arr1r[0]))
# newArr = applyFilters(arr1r, np.zeros((len(arr1r), len(arr1r[0]))), filters[0])
# print(len(newArr))
# print(len(newArr[0]))

In [27]:
def forward(img):
    outputLayers = [[img]]
    for num in range(layerNum):
        outputLayers.append([])
        for fil in range(filterNum):
            newImage = np.zeros((len(outputLayers[num][fil]), len(outputLayers[num][fil][0])))
            applyFilters(outputLayers[num][fil], newImage, filters[filterNum*num + fil])
            print(filterNum*num + fil)
            print(newImage)
            outputLayers[num+1].append(np.divide(newImage, filterNum))
    return outputLayers
out = forward(arr1r)
print(out)

0
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.0032029   0.00592836 ...,  0.08490688  0.08490688
   0.08490688]
 [ 0.          0.0061529   0.00920913 ...,  0.16853163  0.16853163
   0.16853163]
 ..., 
 [ 0.          0.07561992  0.13468652 ...,  0.71912592  0.71912592
   0.71912592]
 [ 0.          0.07313612  0.12799114 ...,  0.69803922  0.69803922
   0.69803922]
 [ 0.          0.07313612  0.12799114 ...,  0.69803922  0.69803922
   0.69803922]]
1
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   4.80793883e-05 ...,   9.11204198e-03
    9.11204198e-03   9.11204198e-03]
 ..., 
 [  0.00000000e+00   0.00000000e+00   9.36497551e-03 ...,   8.78248931e-01
    8.78248931e-01   8.78248931e-01]
 [  0.00000000e+00   0.00000000e+00   9.067

In [ ]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

In [28]:
error = 1-np.mean(out[len(out)-1])
# def back(img, startlayer, error):
#     print(startlayer)
#     newout = np.zeros((len(out[startlayer-1]), len(out[startlayer-1][0])))
#     if startlayer == -1:
#         return out[0]
#     delta = applyFilters(back(img, startlayer-1, error),
#                  newout,
#                  np.multiply(np.rot90(np.rot90(filters[startlayer])),activation(img,deriv=True)[:filterSize,:filterSize]))
#     filters[startlayer]=np.add(filters[startlayer],array(delta[:filterSize,:filterSize]))
#     print(startlayer)
#     return newout
def back(layer, grad):
    if layer==0:
        return
    print(layer)
    #keep array of all gradients/changes/deltas
    new_grad = np.zeros((filterNum,filterSize,filterSize), dtype=np.float64)
    for x in range(filterNum):
        r = 0
        for row in filters[filterNum*layer+x-1]:
            c=0
            for w in row:
                new_grad[x][r][c]=np.sum(np.multiply(np.multiply(np.multiply(out[layer-1], w), grad[x][r][c]),error))
                filters[filterNum*layer+x-1][r][c]+=new_grad[x][r][c]
                c+=1
            r+=1
    back(layer-1, new_grad)
backout = back(layerNum, np.ones((filterNum,filterSize,filterSize)))
#take all output from prev. layer, multiply by corresponding gradient = new deltas 

6
5
4
3
2
1


In [29]:
print(filters)

[[[  6.97773730e+08   3.49981194e+10   1.09834766e+09   8.77750048e+09
     3.16510635e+10   1.46555300e+11   1.73887496e+09   2.55952286e+10
     2.51550107e+11   2.43184101e+10]
  [  3.46280934e+11   2.05942270e+10   7.23029989e+10   1.05731905e+10
     1.18066799e+10   3.33071338e+08   3.04269087e+09   5.01609839e+09
     3.58305217e+10   6.30792618e+09]
  [  3.04530415e+11   9.12535980e+09   2.26703545e+08   1.65742595e+11
     9.62762060e+11   4.41519075e+09   2.60531575e+10   4.44841419e+10
     1.18262548e+11   1.75033918e+11]
  [  2.39590266e+09   9.86358752e+10   9.53023270e+10   4.58251865e+07
     7.61327580e+10   4.82522913e+11   7.44442680e+09   3.47511069e+11
     2.52385341e+11   5.62620212e+08]
  [  3.83964771e+09   4.49910192e+08   1.78301712e+11   4.52783031e+09
     3.03770230e+10   9.69786759e+09   1.96340017e+08   3.79673622e+11
     1.53639944e+10   1.22606179e+11]
  [  6.83636607e+10   3.48253537e+10   6.86705958e+10   2.55170299e+10
     5.61459811e+10   1.26813

In [ ]:
for x in range(epochs):
    out = forward(arr1r)
    error = 1-np.mean(out[len(out)-1])
    print(error)
    back(layerNum, np.ones((filterNum,filterSize,filterSize)))

0
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   8.05203820e-06   7.76770698e-04 ...,   4.09035369e-02
    4.09035369e-02   4.09035369e-02]
 [  0.00000000e+00   1.57176538e-03   2.94585294e-03 ...,   1.88740154e-01
    1.88740154e-01   1.88740154e-01]
 ..., 
 [  0.00000000e+00   3.51952721e-02   9.77285024e-02 ...,   7.09735822e-01
    7.09735822e-01   7.09735822e-01]
 [  0.00000000e+00   3.46555125e-02   9.16054596e-02 ...,   6.98039216e-01
    6.98039216e-01   6.98039216e-01]
 [  0.00000000e+00   3.46555125e-02   9.16054596e-02 ...,   6.98039216e-01
    6.98039216e-01   6.98039216e-01]]
1
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.53830387e-10 ...,   2.10649854e-03
    2.10649854e-0